## Train

In [ ]:
import os
import json
import torch
from logger import logger
from ultralytics import YOLO


# Check if ROCm-compatible GPU is available
logger.info(f"PyTorch version: {torch.__version__}")
logger.info(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    logger.info(f"Device count: {torch.cuda.device_count()}")
    logger.info(f"Current device index: {torch.cuda.current_device()}")
    logger.info(f"Device name: {torch.cuda.get_device_name(0)}")
    logger.info(f"Device capability: {torch.cuda.get_device_capability(0)}")
else:
    logger.warning(f"No compatible GPU detected use CPU.")

with open("config.json", "r") as f:
    config = json.load(f)
os.chdir(config["workdir"])
model = YOLO("models/yolo11s.pt")
model.train(data="yolo-config.yaml", epochs=50, imgsz=960, device="0", batch=8)
model.save("models/yolo11s-wow-fishing-classic.pt")
logger.info(f"Done.")

## Train-Val-Test Split

In [3]:
import random
import shutil
import json
import os
from pathlib import Path
from logger import logger

with open("config.json", "r") as f:
    config = json.load(f)
os.chdir(config["workdir"])

# Configuration
DATA_DIR = Path("datasets/wow-fishing-classic")
IMAGE_DIR = DATA_DIR / "images"
LABEL_DIR = DATA_DIR / "labels"
SPLITS = {"train": 0.8, "val": 0.1, "test": 0.1}
IMG_EXTENSIONS = [".jpg", ".jpeg", ".png"]

# Create destination folders
for split in SPLITS:
    (IMAGE_DIR / split).mkdir(parents=True, exist_ok=True)
    (LABEL_DIR / split).mkdir(parents=True, exist_ok=True)

# Gather all images
image_files = [f for f in IMAGE_DIR.glob("*") if f.suffix.lower() in IMG_EXTENSIONS]
random.shuffle(image_files)

# Calculate split sizes
total = len(image_files)
n_train = int(SPLITS["train"] * total)
n_val = int(SPLITS["val"] * total)
n_test = total - n_train - n_val

splits = {
    "train": image_files[:n_train],
    "val": image_files[n_train:n_train + n_val],
    "test": image_files[n_train + n_val:]
}

# Move files
for split_name, files in splits.items():
    for img_path in files:
        label_path = LABEL_DIR / img_path.with_suffix(".txt").name
        shutil.move(str(img_path), str(IMAGE_DIR / split_name / img_path.name))
        if label_path.exists():
            shutil.move(str(label_path), str(LABEL_DIR / split_name / label_path.name))

logger.info("✅ Dataset successfully split into train/val/test")


[10:10:41] [INFO ] [3786104586.py:48]  ✅ Dataset successfully split into train/val/test
